In [14]:
# !pip install -U --user /kaggle/input/lama-nn/LightAutoML-0.3.7.4-py3-none-any.whl >> none

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
import copy
import gc
import os
import time
import lightautoml

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
# target = pd.read_csv("tr.csv", usecols=['user_id', 'is_male'])
# # test = pd.read_csv("/kaggle/input/mlcupmlfeaturesv6/test_ml_num_v2.csv", nrows=10000)
# # test.sort_values("user_id", inplace=True)

# print(len(target))

In [4]:
# oof_age = pd.read_csv("oof_warm_male_big_automl.csv")
# test_age = pd.read_csv("test_warm_male_big_automl.csv")

# pca_train = pd.read_csv("pca_train.csv")
# pca_test = pd.read_csv("pca_test.csv")

# train_dart_pred = pd.read_csv("train_dart_pred.csv")
# test_dart_pred = pd.read_csv("test_dart_pred.csv")

# fl_train = pd.read_csv("fl_train_11.csv")
# fl_test = pd.read_csv("fl_test_11.csv")

# gru_train = pd.read_csv("ismale_gru_oof.csv")
# gru_test = pd.read_csv("ismale_gru_test.csv")
# gru_train.rename(columns={'is_male':'is_male_gru'}, inplace=True)
# gru_test.rename(columns={'is_male':'is_male_gru'}, inplace=True)

# gru_train_2 = pd.read_csv("ismale_gru_oof (1).csv")
# gru_test_2 = pd.read_csv("ismale_gru_test (1).csv")
# gru_train_2.rename(columns={'is_male':'is_male_gru_2'}, inplace=True)
# gru_test_2.rename(columns={'is_male':'is_male_gru_2'}, inplace=True)

# lstm_train = pd.read_csv("ismale_lstm_oof.csv")
# lstm_test = pd.read_csv("ismale_lstm_test.csv")
# lstm_train.rename(columns={'is_male':'is_male_lstm'}, inplace=True)
# lstm_test.rename(columns={'is_male':'is_male_lstm'}, inplace=True)

In [5]:
# oof_age = oof_age.merge(train_dart_pred, on='user_id')
# test_age = test_age.merge(test_dart_pred, on='user_id')

# oof_age = oof_age.merge(pca_train, on='user_id')
# test_age = test_age.merge(pca_test, on='user_id')

# oof_age = oof_age.merge(fl_train, on='user_id')
# test_age = test_age.merge(fl_test, on='user_id')

# oof_age = oof_age.merge(gru_train, on='user_id', how ='left')
# test_age = test_age.merge(gru_test, on='user_id', how ='left')

# oof_age = oof_age.merge(lstm_train, on='user_id', how ='left')
# test_age = test_age.merge(lstm_test, on='user_id', how ='left')

# oof_age = oof_age.merge(gru_train_2, on='user_id', how ='left')
# test_age = test_age.merge(gru_test_2, on='user_id', how ='left')

# oof_age.head()

In [6]:
# oof_age = oof_age.merge(target, on='user_id')
# oof_age = oof_age[~oof_age['is_male'].isna() &  oof_age['is_male'] != 'NA']
# print(len(oof_age), len(test_age))

In [3]:
# # oof_age.to_csv("b/oof_ismale_st_lvl2.csv", index=False)
# test_age.to_csv("b/test_ismale_st_lvl2.csv", index=False)

target = pd.read_parquet("public_train.pqt")
oof_age = pd.read_csv("cold_male/oof_cold_male_big_automl.csv")
test_age = pd.read_csv("cold_male/test_cold_male_big_automl.csv")
oof_age.shape

(264326, 11)

In [4]:
test_age.head()

,user_id,Lvl_0_Pipe_0_Mod_0_LinearL2,Lvl_0_Pipe_1_Mod_0_LightGBM,Lvl_0_Pipe_1_Mod_1_CatBoost,Lvl_0_Pipe_1_Mod_2_Tuned_CatBoost,Lvl_0_Pipe_2_Mod_0_TorchNN_0,Lvl_0_Pipe_2_Mod_1_Tuned_TorchNN_resnet_tuned_1,Lvl_0_Pipe_2_Mod_2_TorchNN_resnet_2,Lvl_0_Pipe_2_Mod_3_TorchNN_dense_3,Lvl_0_Pipe_2_Mod_4_TorchNN_denselight_4,Lvl_0_Pipe_2_Mod_5_Tuned_TorchNN_denselight_tuned_5
0,6,0.495247,0.470335,0.474907,0.477269,0.512303,0.473377,0.455029,0.462456,0.459163,0.463749
1,7,0.719520,0.755103,0.761836,0.774084,0.617500,0.656957,0.696093,0.684056,0.698320,0.716735
2,9,0.506794,0.485184,0.470617,0.471027,0.454835,0.510247,0.495137,0.489315,0.504273,0.508345
3,10,0.546880,0.575170,0.564337,0.560174,0.482871,0.547139,0.538659,0.537895,0.537746,0.568955
4,11,0.560733,0.563391,0.602668,0.627738,0.498907,0.541253,0.558168,0.550799,0.556547,0.541747


In [5]:
oof_age = oof_age.merge(target, on='user_id')
oof_age.shape

(264326, 13)

In [5]:
# train = train[~train['is_male'].isna() & (train['is_male'].apply(lambda x: x != 'NA'))]
# train['is_male'] = train['is_male'].astype('int')
# print(len(train))

In [7]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.tasks import Task
import time


roles = {'target': 'is_male',
                 'drop': ['user_id', 'index', "age"]}
default_nn_params = {
    "n_epochs": 200,
}
default_nn_params_2 = {
    "n_epochs": 100,
}
automl = TabularAutoML(
    task = Task(name='binary'),
    cpu_limit = 24,
    timeout=3600*1500,
    gpu_ids='0',
    debug=True,
    general_params = {"use_algos": [["linear_l2"]]},
    nn_pipeline_params = {"use_te": True,  "max_intersection_depth": 0},
    reader_params = {'cv': 10, 'random_state': 44},
)
start_time = time.time()
oof_pred = automl.fit_predict(
        oof_age,
        roles=roles,
        verbose = 1
    )
end_time = time.time()

[19:53:20] Stdout logging level is INFO.
[19:53:20] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[19:53:20] Task: binary

[19:53:20] Start automl preset with listed constraints:
[19:53:20] - time: 5400000.00 seconds
[19:53:20] - CPU: 24 cores
[19:53:20] - memory: 16 GB

[19:53:20] Train data shape: (264326, 13)

[19:53:22] Layer 1 train process start. Time left 5399998.07 secs
[19:53:22] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[19:53:25] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.6221888753508898
[19:53:25] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[19:53:25] Time left 5399995.05 secs

[19:53:25] Layer 1 training completed.

[19:53:25] Automl preset training completed in 4.95 seconds

[19:53:25] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (10 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) 



In [8]:
print(f"spend time: {end_time - start_time}")
print(automl.create_model_str_desc())

spend time: 4.977286338806152
Final prediction for new objects (level 0) = 
	 1.00000 * (10 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) 


In [9]:
test_pred_ismale = automl.predict(test_age)
sub = pd.DataFrame()
sub['user_id'] = test_age.user_id
sub['is_male'] = test_pred_ismale.data[: ,0]
sub.sort_values("user_id", inplace=True)
sub.to_csv("stack_lvl2_ismale_final_cold.csv", index=False)

sub.head()

,user_id,is_male
0,6,0.541119
1,7,0.253902
2,9,0.516075
3,10,0.424122
4,11,0.416187


In [15]:
# 0.8985054971831067 final